In [17]:
from IPython.display import display
import pandas as pd
import numpy as np

# Import seaborn for statistic evaluation
import seaborn as sns
# Apply the default theme
sns.set_theme()
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
import sklearn
import re
import ipywidgets as widgets
from IPython.display import display, clear_output
from contextlib import contextmanager
import os, sys

from sklearn.model_selection import train_test_split

In [18]:
book_path = r'C:\Users\Legion\Downloads\b\1INTRO2AI\BX-Books.csv'
user_path = r'C:\Users\Legion\Downloads\b\1INTRO2AI\BX-Users.csv'
rating_path = r'C:\Users\Legion\Downloads\b\1INTRO2AI\BX-Book-Ratings.csv'
books = pd.read_csv(book_path, encoding='latin-1' , on_bad_lines='skip' , sep=';' , low_memory=False, escapechar='\\')
users = pd.read_csv(user_path, encoding='latin-1' , on_bad_lines='skip' , sep=';' , low_memory=False, escapechar='\\')
ratings = pd.read_csv(rating_path, encoding='latin-1' , on_bad_lines='skip' , sep=';' , low_memory=False, escapechar='\\')

In [19]:
books.dtypes

ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication     int64
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object

In [20]:
pd.set_option('display.max_colwidth', None)

In [21]:
users.dtypes

User-ID       int64
Location     object
Age         float64
dtype: object

In [43]:
ratings.dtypes

User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object

In [23]:
# Data Preprocessing

books.loc[books['Book-Author'].isnull(), 'Book-Author'] = "No author"
books.loc[books['Publisher'].isnull(), 'Book-Author'] = "Other"

In [24]:
print(users.shape)
users.head()
print(sorted(users['Age'].unique()))

(278858, 3)
[nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 118.0, 119.0, 123.0, 124.0, 127.0, 128.0, 132.0, 133.0, 136.0, 137.0, 138.0, 140.0, 141.0, 143.0, 146.0, 147.0, 148.0, 151.0, 152.0, 156.0, 157.0, 159.0, 162.0, 168.0, 172.0, 175.0, 183.0, 186.0, 189.0, 199.0, 200.0, 201.0, 204.0, 207.0, 208.0, 209.0, 210.0, 212.0, 219.0, 220.0, 

In [25]:
users.loc[(users['Age'] < 5) | (users['Age'] > 100), 'Age'] = pd.NA
users['Age'] = users['Age'].fillna(users['Age'].mean())
users['Age'] = users['Age'].astype(np.int64)

In [48]:
# print(users.shape[0] * books.shape[0])
# Check if people rated is in database
# r_existpeople = ratings[ratings['User-ID'].isin(users['User-ID'])]
# print(r_existpeople.shape)

# Check if books rated is in database
r_existbook = ratings[ratings['ISBN'].isin(books['ISBN'])]
r_existbook

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149774,276704,0876044011,0
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10


In [27]:
ratings['Book-Rating'].unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2], dtype=int64)

In [52]:
ratings_nonzero = r_existbook[r_existbook['Book-Rating'] != 0]
# ratings_explicit
ratings_allzero = r_existbook[r_existbook['Book-Rating'] == 0]
# ratings_implicit
ratings_nonzero

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
8,276744,038550120X,7
16,276747,0060517794,9
...,...,...,...
1149771,276704,0743211383,7
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10


In [81]:
# reducing to user rating 100 books and books have 100 ratings
used_rating = ratings_nonzero

counts1 = ratings_nonzero['User-ID'].value_counts()
counts = ratings_nonzero['ISBN'].value_counts()
print(counts1)
print(counts)

used_rating = used_rating[used_rating['User-ID'].isin(counts1[counts1 >= 100].index)]
used_rating = used_rating[used_rating['ISBN'].isin(counts[counts >= 50].index)]

print(used_rating)

11676     6943
98391     5691
189835    1899
153662    1845
23902     1180
          ... 
114767       1
114771       1
114772       1
114777       1
276721       1
Name: User-ID, Length: 68092, dtype: int64
0316666343    707
0971880107    581
0385504209    487
0312195516    383
0060928336    320
             ... 
8420431249      1
840804978X      1
8408020854      1
355175537X      1
0806917695      1
Name: ISBN, Length: 149842, dtype: int64
         User-ID        ISBN  Book-Rating
1456      277427  002542730X           10
1474      277427  0061009059            9
1522      277427  0316776963            8
1543      277427  0345413903           10
1578      277427  0385424736            9
...          ...         ...          ...
1146735   275970  038572179X            9
1146737   275970  0385722206           10
1146809   275970  043936213X            9
1146820   275970  0440180295            9
1146852   275970  044651652X            6

[7660 rows x 3 columns]


In [82]:
ratings_matrix = used_rating.pivot(index = 'User-ID', columns = 'ISBN', values = 'Book-Rating')
userID = ratings_matrix.index
ISBN = ratings_matrix.columns
print(ratings_matrix.shape)
ratings_matrix = ratings_matrix.fillna(0)

ratings_matrix

(427, 531)


ISBN,002542730X,0060096195,006016848X,0060173289,0060175400,0060199652,0060391626,0060392452,0060502258,0060915544,...,1558745157,1559029838,1573225517,1573225789,1573227331,1573229326,1573229571,1592400876,1844262553,1878424319
User-ID,,,,,,,,,,,,,,,,,,,,,
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
# KNN Function
#setting global variables
global metric,k
k=10
global_metric='cosine'
global_algorithm = 'brute'

In [84]:
def findksimilarusers(user_id, ratings, metric=global_metric, algo=global_algorithm,k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = algo)
    model_knn.fit(ratings.values)
    loc = ratings.index.get_loc(user_id)
    distances, indices = model_knn.kneighbors(ratings.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1 - distances.flatten()
    
    return similarities, indices

In [85]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = global_metric, algorithm = global_algorithm, k=k):
    prediction=0
    user_loc = ratings.index.get_loc(user_id)
    item_loc = ratings.columns.get_loc(item_id)
    similarities, indices = findksimilarusers(user_id, ratings, metric, algorithm, k) #similar users based on cosine similarity
    mean_rating = ratings.iloc[user_loc, :].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == user_loc:
            continue;
        else: 
            ratings_diff = ratings.iloc[indices.flatten()[i],item_loc]-np.mean(ratings.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    
    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below
    if prediction <= 0:
        prediction = 1   
    elif prediction >10:
        prediction = 10
    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    # print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [86]:
predict_userbased(11676,'0001056107',ratings_matrix);

KeyError: '0001056107'

In [87]:
def recommendedItem(user_id, ratings, metric=global_metric):
    if (user_id not in ratings.index.values) or type(user_id) is not int:
        print("User id should be a valid integer from this list : \n\n {}".format(re.sub('[\[\]]', '', np.array_str(ratings_matrix.index.values))))
    else:
        prediction = []
        
#         ids = ['Item-based (correlation)', 'Item-based (cosine)', 'User-based (correlation)', 'User-based (cosine)']
#         select = widgets.Dropdown(options = ids, value = ids[0], description = ' Select approach', width = '1000px')
#         def on_change(change):
#             clear_output(wait = True)
#             prediction = []
#             if change['type'] == 'change' and change['name'] == 'value':
#                 if (select.value == 'Item-based (correlation)') | (select.value == 'User-based (correlation)'):
#                     global_metric = 'correlation'
#                 else:
#                     global_metric = 'cosine'
                    
#                 with suppress_stdout():
#         if (select.value == 'User-based (correlation)') | (select.value == 'User-based (cosine)'):

        total = ratings.shape[1]
        for i in range(ratings.shape[1]):
            if (ratings[str(ratings.columns[i])][user_id] != 0): #not rated already
                print("\r{0:<0.2f}% Finished".format(i*100/total), end='')
                prediction.append(predict_userbased(user_id, str(ratings.columns[i]) ,ratings, metric))
            else:                    
                prediction.append(-1) #for books that user already rated
#         else:
#             print("Item-based not implemented yet")
        print("\r100.00% Finished")
        
        prediction = pd.Series(prediction)
        prediction = prediction.sort_values(ascending = False)
        print("...")
        print(prediction)
        
        recommended = prediction[:10]
        print("\nFor the User-based ({0} - {1}) approach, the following books are recommended: \n".format(global_algorithm, global_metric))
        for i in range(len(recommended)):
            print("{0}. Rated {1}: {2}".format(i+1, prediction.values[i], books['Book-Title'][recommended.index[i]]))
                

In [88]:
print(ratings_matrix)
global_algorithm = 'brute'
global_metric = 'cosine'
recommendedItem(30711, ratings_matrix)

ISBN     002542730X  0060096195  006016848X  0060173289  0060175400  \
User-ID                                                               
2033            0.0         0.0         0.0         0.0         0.0   
2110            0.0         0.0         0.0         0.0         0.0   
2276            0.0         0.0         0.0         0.0         0.0   
4017            0.0         0.0         0.0         0.0         0.0   
4385            0.0         0.0         0.0         0.0         0.0   
...             ...         ...         ...         ...         ...   
274061          0.0         0.0         0.0         0.0         0.0   
274301          0.0         0.0         0.0         0.0         0.0   
275970          0.0         0.0         0.0         0.0         0.0   
277427         10.0         0.0         0.0         0.0         0.0   
278418          0.0         0.0         0.0         0.0         0.0   

ISBN     0060199652  0060391626  0060392452  0060502258  0060915544  ...  \


In [89]:
global_algorithm = 'kd_tree'
global_metric = 'euclidean'
recommendedItem(30711, ratings_matrix)

100.00% Finished
...
247    5
241    4
518    2
180    2
322    2
      ..
173   -1
172   -1
171   -1
170   -1
530   -1
Length: 531, dtype: int64

For the User-based (kd_tree - euclidean) approach, the following books are recommended: 

1. Rated 5: A Judgement in Stone
2. Rated 4: Crazy Horse
3. Rated 2: A Suitable Boy : Novel, A
4. Rated 2: Die Mars- Chroniken. Roman in ErzÃ?Â¤hlungen.
5. Rated 2: The Rivan Codex : Ancient Texts of THE BELGARIAD and THE MALLOREON
6. Rated 1: The Sky Is Falling
7. Rated 1: Der Tod kommt schnell in Mexico. ErzÃ?Â¤hlungen.
8. Rated 1: The Folk of the Faraway Tree
9. Rated 1: Urn Burial
10. Rated 1: Simple Wicca (Simple Wisdom Book)
